# TP - HMMA238

Par Mathias Gout et Julie Røste.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Exercice 1 - Le jeu de la vie

### Question 4:

In [ ]:
from utils import calcul_nb_voisins
from utils import iteration_jeu
Z_init = [[0,0,0,0,0,0], 
          [0,0,0,1,0,0], 
          [0,1,0,1,0,0], 
          [0,0,1,1,0,0],
          [0,0,0,0,0,0], 
          [0,0,0,0,0,0]]
     

calcul_nb_voisins(Z_init)

`calcul_nb_voisins(Z)` affiche le nombre de voisins vivants de chaque cellules

### Question 5:

In [ ]:
Z_init

In [ ]:
def plotJeuDeLaVie(nbIterations,Z,iter_func):
    """
    Plotting the evolution of the matrices of Jeu de La Vie.
    """
    Zcopy = Z.copy()
    plt.figure(figsize=(15,6))
    plt.subplot(2,5,1)
    plt.title("Iteration 0")
    plt.imshow(Zcopy)
    for i in range(2,nbIterations+1):
        plt.subplot(2,5,i)
        Zcopy = iter_func(Zcopy)
        plt.title("Iteration "+str(i-1))
        plt.imshow(Zcopy)

plotJeuDeLaVie(8,np.asarray(Z_init),iteration_jeu)

### Question 6:

On remarque que les cellules vivantes de l'itération 0 sont disposées de la même façon que celles de l'itération 4. 
<br/>Entre l'itération 0 et 4, les cellules vivantes se sont toutes déplacées d'une cellule en bas et d'une cellule à droite.

A partir de l'itération 7, les cellules ne changent plus d'état. 
Les quatres cellules vivantes sont regroupées en bas à droite et ont toutes 3 voisins vivants (aucun décès possible).
<br/>Comme aucune cellule morte n'a 3 voisins vivants (pas de naissance possible), il n'y aura pas d'évolution dans les prochaines itérations.


### Question 7 - Implémentation avec `numba`

In [ ]:
from numba import jit

Comments: Have to rewrite functions to numpy, it not it didn't work..... So they are rewritten in utils to `calcul_np_voisins_np` and `iteration_jeu_np`. 

In [ ]:
Z_init = [[0,0,0,0,0,0], # remark: Using python list
          [0,0,0,1,0,0], 
          [0,1,0,1,0,0], 
          [0,0,1,1,0,0],
          [0,0,0,0,0,0], 
          [0,0,0,0,0,0]]
Z_np = np.array(Z_init)

In [ ]:
import time
from utils import calcul_nb_voisins_np
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
print(calcul_nb_voisins_np(Z_np))
end = time.time()
print("Elapsed (including compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE.
start = time.time()
print(calcul_nb_voisins_np(Z_np))
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

# NOT USING NUMBA.JIT() TO COMPARE RUNNING TIMES. 
start = time.time()
print(calcul_nb_voisins(Z_init))
end = time.time()
print("Elapsed (not using numba.jit()) = %s" % (end-start))

On a la même matrice qu'en question 4, mais c'est assez efficace de ne pas utiliser `numba`. Pour comparer le temps utilisé pour question 5, on fait encore une fois le protocole.

In [ ]:
from utils import iteration_jeu_np
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
plotJeuDeLaVie(10,Z_np,iteration_jeu_np)
end = time.time()
print("Elapsed (including compilation) = %s" % (end - start))

In [ ]:
# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE.
start = time.time()
plotJeuDeLaVie(10,Z_np,iteration_jeu_np)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
# NOT USING NUMBA.JIT() TO COMPARE RUNNING TIMES. 
start = time.time()
plotJeuDeLaVie(10,Z_np,iteration_jeu)
end = time.time()
print("Elapsed (not using numba.jit()) = %s" % (end-start))

On ne voit pas que c'est plus efficace à utiliser jit dans ce cas là.

### Question 8

In [ ]:
def plotJeuDeLaVie_q8(nbIterations,Z,iter_func):
    """
    Plotting the evolution of the matrices of Jeu de La Vie.
    """
    Zcopy = Z.copy()
    plt.figure(figsize=(20,10))
    plt.subplot2grid((6,5), (0,0)) # Have to use subplot2grid instead of subplot for > 10 subplots.
    plt.title("Iteration 0")
    plt.imshow(Zcopy)
    for i in range(6):
        for j in range(5):
            if ((i*5+j)>=nbIterations):
                break
            plt.subplot2grid((6,5), (i,j))
            Zcopy = iter_func(Zcopy)
            plt.title("iteration "+str(i*5+j))
            plt.imshow(Zcopy)


In [ ]:
from ipywidgets import interact,fixed

interact(plotJeuDeLaVie_q8, nbIterations=(0,30,1),Z=fixed(Z_np),iter_func = fixed(iteration_jeu_np))

In [ ]:
n = 100
Z_huge = np.zeros((n,n))
Z_huge[10:16,10:16] = Z_np

In [ ]:
plotJeuDeLaVie_q8(7,Z_huge,iteration_jeu_np)

In [ ]:
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
calcul_nb_voisins_np(Z_huge)
end = time.time()
print("Elapsed (including compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
calcul_nb_voisins_np(Z_huge)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

# 
start = time.time()
calcul_np_voisins(Z_huge)
end = time.time()
print("Elapsed (not using numba.jit()) = %s" % (end-start))